# plot orientation in real time

In [1]:
import math
import serial
from threading import Thread
import time
import re

import pylab as pl
from IPython import display
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
# connect to the Serial
ser = serial.Serial('COM3', 9600)
time.sleep(2)

In [3]:
def parse_serial(serial_msg):
    """
    Function to parse serial data to extract float values
    format 'x:19.34 y:23.01 z:-33.83' to x, y, z float values

    Args:
        - (str) string with format 'x:19.34 y:23.01 z:-33.83'
    Return:
        - (list) x, y, z float values
    """
    xyz_list = re.findall('[-+]?[0-9]*\.?[0-9]*', serial_msg)
    return [float(i) for i in filter(lambda item: item, xyz_list)]

In [4]:
# initialize x orientation
x_orientation = 0

# 119 got from Arduino with IMU.gyroscopeSampleRate();
gyroscope_sample_rate = 119

def UpdateOrientation(ser):
    """
    Function to integration the x data from the Gyroscope
    and update the global variable x_orientation with the new value

    Args:
        - (serial.Serial) serial to get the gyroscope data
    """
    global x_orientation

    while True:
        serial_msg_bytes = ser.readline()
        serial_msg = serial_msg_bytes.decode()
        dx, dy, dz = parse_serial(serial_msg)
        
        # The gyroscope values are in degrees-per-second
        # divide each value by the number of samples per second
        dx_normalized = dx / gyroscope_sample_rate;

        # remove noise
        if(abs(dx_normalized) > 0.004):
            # update orientation
            x_orientation = x_orientation - dx_normalized*1.25
            x_orientation = x_orientation%360

In [5]:
# run the thread to update the x orientation in real time
Thread(target=UpdateOrientation, args=(ser,)).start()

In [ ]:
while True:
    plt.clf()

    # plot origin in blue
    plt.scatter(0, 0, s=100, c='b')

    # find position of the end of the needle
    # 0.1 far from origin in direction of the orientation
    distance = 0.1
    x_orientation_rad = math.radians(x_orientation)
    x_pos = math.cos(x_orientation_rad)*distance
    y_pos = math.sin(x_orientation_rad)*distance

    # plot line between both position with circles
    plt.plot([0, x_pos], [0, y_pos], 'ro-')
    
    plt.xlim([-distance, distance])
    plt.ylim([-distance, distance])
    plt.pause(0.1)